In [1]:
import os
import numpy as np
import yaml
import pprint
import matplotlib.pyplot as plt
import utils.signals.gps_l1ca as gps_l1ca
import utils.sample_processing.sample_streaming as sample_streaming
import utils.sample_processing.bpsk_acquisition as bpsk_acquisition
import utils

In [2]:
# Get local-data directory for storing downloaded data files
data_dir = os.path.join(os.path.dirname(os.path.dirname(utils.__file__)), "local-data")
os.makedirs(data_dir, exist_ok=True)
raw_collects_dir = os.path.join(data_dir, "raw-collects")
os.makedirs(raw_collects_dir, exist_ok=True)
raw_collect_filenames = sorted(os.listdir(raw_collects_dir))
print(f"Data directory: {data_dir}")
collect_metadata_filepath = os.path.join(raw_collects_dir, "metadata.yml")
with open(collect_metadata_filepath, "r") as f:
    collect_metadata = yaml.safe_load(f)
# pprint.pprint(collect_metadata)
collect_filepaths = {
    collect_id: os.path.join(raw_collects_dir, collect_info["filename"])
    for collect_id, collect_info in collect_metadata["collections"].items()
}

samp_rate = collect_metadata["samp_rate"]
inter_freq_l1_hz = collect_metadata["bands"]["L1"]["inter_freq"]
sample_params_dict = collect_metadata["sample_params"]
sample_params = sample_streaming.SampleParameters.from_dict(sample_params_dict)
print(f"Sample rate: {samp_rate/1e6} MHz")
# pprint.pprint(sample_params_dict)
pprint.pprint(sample_params)

collect_id_list = sorted(collect_filepaths.keys())
print("Available collects:")
for collect_id in collect_id_list:
    print(f"  {collect_id}: {os.path.basename(collect_filepaths[collect_id])}")

collect_id = collect_id_list[0]
collect_filepath = collect_filepaths[collect_id]
print(f"Using collect ID: {collect_id}")
# print(f"  Filepath: {collect_filepath}")

Data directory: /home/brianbw/projects/fall-2025-lectures/local-data
Sample rate: 25.0 MHz
SampleParameters(bit_depth=4,
                 is_complex=True,
                 is_integer=True,
                 is_signed=True,
                 is_i_lsb=True)
Available collects:
  20220513_173218_USRP2: CO_A2_20220513_173218_000211_G1_B1_USRP2.sc4
Using collect ID: 20220513_173218_USRP2


In [7]:
# Load acquisition results from file
import pickle
acq_results_directory = os.path.join(data_dir, "acquisition-results")
os.makedirs(acq_results_directory, exist_ok=True)
acq_results_version_id = "v1"
acq_results_filepath = os.path.join(
    acq_results_directory, f"{collect_id}.{acq_results_version_id}.pkl"
)
with open(acq_results_filepath, "rb") as f:
    acq_results = pickle.load(f)

acquired_signal_ids = sorted(list(filter(
    lambda sig_id: acq_results[sig_id].signal_detected, acq_results.keys()
)))
print(f"Acquired signals in collect {collect_id}: ")
print(", ".join(acquired_signal_ids))

Acquired signals in collect 20220513_173218_USRP2: 
G01, G02, G03, G06, G11, G13, G14, G17, G19, G24, G28, G30


In [15]:
sig_id = "G01"
acq_result: bpsk_acquisition.AcquisitionResult = acq_results[sig_id]
print(f"Acquisition result for signal {sig_id}:")
acq_doppler_hz = acq_result.acq_doppler_hz
acq_code_phase_seconds = acq_result.acq_code_phase_seconds
print(f"  Detected: {acq_result.signal_detected}")
print(f"  Doppler (Hz): {acq_doppler_hz}")
print(f"  Code Phase (secs): {acq_code_phase_seconds:.6f}")
print(f"  Correlation Peak Value: {acq_result.normalized_peak_value:.3f}")

prn = int(sig_id[1:3])
code_seq = gps_l1ca.get_GPS_L1CA_code_sequence(prn)
code_rate_chips_per_sec = gps_l1ca.CODE_RATE
code_length_chips = gps_l1ca.CODE_LENGTH

# Define code phase offsets for early, prompt, and late correlators
early_chip_offset = 0.5  # chips
chip_bin_delta = -0.5
num_chip_bins = 3
chip_bin_offsets = early_chip_offset + chip_bin_delta * np.arange(num_chip_bins)

Acquisition result for signal G01:
  Detected: True
  Doppler (Hz): -3000.0
  Code Phase (secs): 0.001199
  Correlation Peak Value: 110.764


In [ ]:
buffer_duration_ms = 40
block_duration_ms = 1
buffer_size_samples = int(samp_rate * buffer_duration_ms / 1e3)
block_size_samples = int(samp_rate * block_duration_ms / 1e3)

from dataclasses import dataclass
@dataclass
class TrackingSignalState:
    code_phase_seconds: float
    carrier_phase_cycles: float
    doppler_freq_hz: float

signal_state = TrackingSignalState(
    code_phase_seconds=acq_code_phase_seconds,
    carrier_phase_cycles=0.0,
    doppler_freq_hz=acq_doppler_hz,
)

with sample_streaming.FileSampleStream(
        collect_filepath,
        sample_params,
        buffer_size_samples,
        block_size_samples,
    ) as sample_stream:

    sample_block_generator = sample_stream.sample_block_generator()
    
    for i_block, sample_block in enumerate(sample_block_generator):
        
        # Mixdown to baseband (in-place)
        sample_streaming.mixdown_samples(
            sample_block,
            sample_block,
            samp_rate,
            # A mistake I once made was neglecting to account for the
            # phase shift when mixing down non-zero IF.
            # Usually, the IF frequency is integer-valued such that
            # the phase shift is nominally zero for integer-ms blocks.
            # However, to be robust to non-integer IF frequencies,
            # we include the phase shift here.
            i_block * block_duration_ms * 1e-3 * inter_freq_l1_hz,
            inter_freq_l1_hz,
        )

        # Perform correlation
        replica_code_phase = (
            signal_state.code_phase_seconds + 
        )
        signal_replica = 